In [1]:
!git clone https://github.com/ultralytics/yolov3
!cd /content/drive/MyDrive
!pip install -r requirements.txt

fatal: destination path 'yolov3' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import torch
import csv
# Define the Parser class
class Parser():
      # Initialize the class with a PyTorch model as input
    def __init__(self, model):
     # Store the model as an instance variable
        self.model = model
 # Initialize an empty list to store information about the layers
        self.layers_info = []

    def parse(self):
        # Iterate over all the layers of the model
        for name, module in self.model.named_modules():
          try:
            if isinstance(module, torch.nn.Conv2d):
                layer_type = "Conv2D"
                input_channels = module.in_channels
                output_channels = module.out_channels
                kernel_size = module.kernel_size
                stride = module.stride
                padding = module.padding
                layer_info = [name, layer_type, input_channels, output_channels, kernel_size, stride, padding]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.Linear):
                layer_type = "Linear"
                input_channels = module.in_features
                output_channels = module.out_features
                layer_info = [name, layer_type, input_channels, output_channels]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.BatchNorm2d):
                layer_type = "BatchNorm2D"
                input_channels = module.num_features
                layer_info = [name, layer_type, input_channels]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.ReLU):
                layer_type = "ReLU"
                layer_info = [name, layer_type]
                self.layers_info.append(layer_info)   
            
            elif isinstance(module, torch.nn.Sequential) and hasattr(module, "bottleneck"):
                layer_type = "Bottleneck"
                layer_info = [name, layer_type]
                self.layers_info.append(layer_info) 

            elif isinstance(module, torch.nn.Upsample):
              layer_type = "Upsample"
              scale_factor = module.scale_factor
              layer_info = [name, layer_type, scale_factor]
              self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.LeakyReLU):
              layer_type = "LeakyReLU"
              negative_slope = module.negative_slope
              layer_info = [name, layer_type, negative_slope]
              self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.Dropout):
               layer_type = "Dropout"
               p = module.p
               layer_info = [name, layer_type, p]
               self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.Sequential) and hasattr(module, "retinanet"):
              layer_type = "RetinaNet"
              layer_info = [name, layer_type]
              self.layers_info.append(layer_info)   
              
            elif isinstance(module, torch.nn.ModuleList) and hasattr(module, "yolo"):
              layer_type = "YOLO"
              layer_info = [name, layer_type]
              self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.AdaptiveAvgPool2d):
                layer_type = "AdaptiveAvgPool2d"
                output_size = module.output_size
                layer_info = [name, layer_type, output_size]
                self.layers_info.append(layer_info)
                
            elif isinstance(module, torch.nn.AdaptiveMaxPool2d):
                layer_type = "AdaptiveMaxPool2d"
                output_size = module.output_size
                layer_info = [name, layer_type, output_size]
                self.layers_info.append(layer_info)
                
            elif isinstance(module, torch.nn.Tanh):
                layer_type = "Tanh"
                layer_info = [name, layer_type]
                self.layers_info.append(layer_info)
                
            elif isinstance(module, torch.nn.Sigmoid):
                layer_type = "Sigmoid"
                layer_info = [name, layer_type]
                self.layers_info.append(layer_info)
                
            elif isinstance(module, torch.nn.Softmax):
                layer_type = "Softmax"
                dim = module.dim
                layer_info = [name, layer_type, dim]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.MaxPool2d) or isinstance(module, torch.nn.AvgPool2d):
                if isinstance(module, torch.nn.MaxPool2d):
                    layer_type = "MaxPool2D"
                else:
                    layer_type = "AvgPool2D"
                kernel_size = module.kernel_size
                stride = module.stride
                padding = module.padding
                layer_info = [name, layer_type, kernel_size, stride, padding]
                self.layers_info.append(layer_info)
                
            else:
              layer_type = "Other"
              layer_command = module
              layer_info = [name, layer_type, input_channels, output_channels, kernel_size, stride, padding]
              self.layers_info.append(layer_info)
          except:
            layer_info = [name, layer_command]
            self.layers_info.append(layer_info)

    def write_to_csv(self, file_path):
      # Write the layer information to a CSV file
      with open(file_path, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Type", "Input Channels", "Output Channels", "Kernel Size", "Stride", "Padding"])
        writer.writerows(self.layers_info)
        for info in self.layers_info:
          if len(info) == 4:
            writer.writerow([info[0], info[1], info[2], info[3], "", "", "", ""])
          elif len(info) == 3:
            writer.writerow([info[0], info[1], info[2], "", "", "", "", ""])
          elif len(info) == 7:
            writer.writerow([info[0], info[1], info[2], info[3], info[4], info[5], info[6], ""])
          elif len(info) == 8:
            writer.writerow([info[0], info[1], info[2], info[3], info[4], info[5], info[6], info[7]])
          else:
            writer.writerow([info[0], info[1], "", "", "", "", "", ""])

    

           


In [12]:
import torchvision.models as models

# Instantiate the ResNet50 model
resnet50 = models.resnet50(pretrained=True)

# Instantiate the ModelParser class and pass in the model
parser = Parser(resnet50)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('resnet50_final_2_parsed.csv')


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
import torchvision.models as models

# Instantiate the ResNet50 model
# Load the pre-trained SSD-Large model
ssd_large = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math='fp16')
# Instantiate the ModelParser class and pass in the model
parser = Parser(ssd_large)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('ssd_final_2_large.csv')


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [14]:
import torch
from transformers import BertForSequenceClassification, BertConfig
import pandas as pd

# Load the pre-trained BERT-768 model
Bert = BertForSequenceClassification.from_pretrained("bert-base-uncased")
# Instantiate the ModelParser class and pass in the model
parser = Parser(Bert)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('Bert_final_2.csv')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
import torch
import torchvision
import pandas as pd

# Load RetinaNet model
retinanet = torchvision.models.detection.retinanet.retinanet_resnet50_fpn(pretrained=True)

parser = Parser(retinanet)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('retinanet_final_2.csv')



/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
import torch

# Model
yolo = torch.hub.load('ultralytics/yolov3', 'yolov3')  # or yolov3-spp, yolov3-tiny, custom


Using cache found in /root/.cache/torch/hub/ultralytics_yolov3_master


In [17]:
parser = Parser(yolo)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('yolo_final_2.csv')
